In [2]:
import pysam
import difflib
from typing import NamedTuple

In [3]:
REF_FASTA = '../insilico-builder/test_data/ref_38/hg38.fa'

In [4]:
fasta = pysam.FastaFile(REF_FASTA)

In [5]:
# ref = 'TCAGTTACAGGGTGGGT'
# alt = 'TAAAC'
# pos = 16459345
# chrom = '10'
# ref = 'TCCCTATGCAGAGTAAGGGGAGATGGAAGAAAC'
# alt = 'TTCTGATGGGGGAGATGGAAGAACA'
# pos = 133662439
# chrom = '10'
# ref = 'GAGCG'
# alt = 'GTT'
# pos = 116367644
# chrom = '10'
# ref = 'TTTCAGATATAT'
# alt = 'TCAAATATA'
# pos = 54703779
# chrom = '11'
ref = 'GTA'
alt = 'G'
pos = 242393454
chrom = '1'
# ref = 'GAAAA'
# alt = 'GGAT'
# pos = 116518035
# chrom = '10'

padding = 50

import variant_normalizator
import importlib
importlib.reload(variant_normalizator)

normalizator = variant_normalizator.VariantNormalizator(REF_FASTA)

variants = normalizator.normalize(chrom, pos, ref, alt)
for variant in variants:
    print(variant)

GTGTATATATATGAGTATATATATGTGTATATATATGAGTATATATATGTGTATATATATGAGTATATATATGTGTATATATGAGTATATATATGTGTATATA
GTGTATATATATGAGTATATATATGTGTATATATATGAGTATATATATGTGTATATATGAGTATATATATGTGTATATATGAGTATATATATGTGTATATA
- G|- T|- G|- T|- A|- T|- A|- T|- A|- T|- A|- T|- G|- A|- G|- T|- A|- T|- A|- T|- A|- T|- A|- T|  G|  T|  G|  T|  A|  T|  A|  T|  A|  T|  A|  T|  G|  A|  G|  T|  A|  T|  A|  T|  A|  T|  A|  T|  G|  T|  G|  T|  A|  T|  A|  T|  A|  T|  A|  T|  G|  A|  G|  T|  A|  T|  A|  T|  A|  T|  A|  T|  G|  T|  G|  T|  A|  T|  A|  T|  A|  T|  G|  A|  G|  T|  A|  T|  A|  T|  A|  T|  A|  T|  G|  T|  G|  T|  A|  T|  A|  T|  A|+ T|+ G|+ A|+ G|+ T|+ A|+ T|+ A|+ T|+ A|+ T|+ A|+ T|+ G|+ T|+ G|+ T|+ A|+ T|+ A|+ T|+ A
1	242393403	GTGTATATATATGAGTATATATAT	
1	242393456	A	ATATATATGAGTATATATATGTG


In [6]:
FILE = 'tests/only_pass.vcf'
OUT_FILE = 'tests/only_pass_plain.vcf'

with open(FILE, 'r') as f:
    with open(OUT_FILE, 'w') as f_out:
        for line in f:
            chrom, pos, _, ref, alt = line.strip().split('\t')[:5]
            chrom = chrom.replace('chr', '')
            f_out.write(f'{chrom}\t{pos}\t{ref}\t{alt}\n')